#### import  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import random

import itertools
import time
import statsmodels.api as sm
from sklearn import linear_model
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg

In [ ]:
# dl
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout , Conv2D ,MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3

#### model selection 

In [ ]:
# train data 와 test data로 구분(7:3 비율) X,y : DataFrame
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

In [ ]:
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
scores=[]

for train_idx, val_idx in k_fold.split(X):
#     print(train_idx," : ",val_idx)

    # split train, validation set
    xtr = X.iloc[train_idx]
    ytr = y.iloc[train_idx]
    xte = X.iloc[val_idx]
    yte = y.iloc[val_idx]
    lr = lrg()
    lr.fit(xtr,ytr)
    score = lr.score(xte,yte)
    scores.append(score)

#### linear regression 

In [ ]:
from sklearn.linear_model import LinearRegression
lr0 = LinearRegression()
lr0.fit(x_train,y_train)
# lr0.coef_ , lr0.intercept_
sco = lr0.score(x_test,y_test)
lr0.predict(x_test)

#### time check

In [ ]:
from time import time as tt
iterate_n = 10
first = tt()
for x in range(iterate_n) :
#     train_err[train_err['user_id'].isin(strange_user_id)]
duration = tt() -first
print("duration : " ,duration)

#### google search api 

In [ ]:
from googleapiclient.discovery import build
import json
my_api_key = "api key"
my_cse_id = "custom search engine id"  
def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res
#구글로 검색하기
search_result = google_search("Python Google Custom Search", my_api_key, my_cse_id)

with open('search_result_all.json', 'w', encoding='utf-8') as f:
    json.dump(search_result, f, ensure_ascii=False, indent=4)

#### scaling & oneHot Encoding 

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [ ]:
# 원핫인코딩(더미 변수)
from sklearn import preprocessing    

label_encoder = preprocessing.LabelEncoder()     # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()   # one hot encoder 생성

onehot_location = label_encoder.fit_transform(df['지역'])
onehot_code = label_encoder.fit_transform(df['코드'])
onehot_type = label_encoder.fit_transform(df['유형'])
onehot_day = label_encoder.fit_transform(df['주야'])

#### decission Tree 

In [ ]:
from sklearn.model_selection import train_test_split as tts
xtr , xte,ytr,yte = tts(x,y,random_state=10,test_size=0.3)
from sklearn.tree import DecisionTreeClassifier as dtc
dtcr =dtc(criterion='entropy', max_depth=5)
dtcr.fit(xtr,ytr)

from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, y_hat)  
print(tree_matrix)
print('\n')

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, y_hat)            
print(tree_report)

#### pca 

In [ ]:
from sklearn.decomposition import PCA
comp_n = 2
pca = PCA(n_components=comp_n) # 주성분을 몇개로 할지 결정
cols = []
for x in range(comp_n):
    cols.append('comp_%d'%x)
printcipalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data=printcipalComponents, columns = cols)

#### folium

In [ ]:
import folium
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan',11:'pink'}

df = itaca
cluster2_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng in zip(df.상호명, df.위도, df.경도):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[1],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[1],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster2_map)
cluster2_map

In [ ]:
import folium
from sklearn import preprocessing
from sklearn import cluster
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan',11:'pink'}

columns_list =['경도','위도']
X3 = itaca.loc[:, columns_list]
x = preprocessing.StandardScaler().fit(X3).transform(X3)

# dbm = cluster.DBSCAN()#eps=0.2, min_samples=5
dbm = cluster.KMeans(init='k-means++', n_clusters=5, n_init=10)
dbm.fit(x)  

itaca['cluster'] = dbm.labels_   

cluster3_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(itaca.상호명, itaca.위도, itaca.경도, itaca.cluster):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster3_map)
cluster3_map

In [ ]:
import folium
from sklearn import preprocessing
from sklearn import cluster
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan',11:'pink'}

columns_list =['경도','위도']
X3 = itaca.loc[:, columns_list]
x = preprocessing.StandardScaler().fit(X3).transform(X3)

# dbm = cluster.DBSCAN()#eps=0.2, min_samples=5
dbm = cluster.KMeans(init='k-means++', n_clusters=5, n_init=10)
dbm.fit(x)  

itaca['cluster'] = dbm.labels_   

cluster3_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(itaca.상호명, itaca.위도, itaca.경도, itaca.cluster):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster3_map)
cluster3_map

In [ ]:
from sklearn.model_selection import StratifiedKFold
# 10개의 파일로 쪼갬
n_fold = 2
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)


# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

model.save('sonar_model.h5')

from keras.models import Sequential, load_model

exsonar = load_model('sonar_model.h5')

#### Outlier Detection 

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
#from sklearn.ensemble import IsolationForest
#from sklearn.svm import OneClassSVM

outlier = LocalOutlierFactor()
#outlier = IsolationForest()
#outlier = OneClassSVM()

y_predict = outlier.fit_predict(df_train.drop(['score'], axis=1))
# print(y_predict) #[ 1  1  1  1  1  1  1 -1  1  1  1]
df_train['outlier'] = y_predict

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping
# 모델의 설정
model = Sequential()
model.add(Dense(30,  input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

# 모델 저장 폴더 설정
MODEL_DIR = './model0/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

# 모델 저장 조건 설정
modelpath=MODEL_DIR+"{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)
# 모델 실행 및 저장
history = model.fit(X, Y, validation_split=0.2, epochs=200, batch_size=200, verbose=0, callbacks=[checkpointer,early_stopping_callback])

# y_vloss에 테스트셋으로 실험 결과의 오차 값을 저장
y_vloss=history.history['val_loss']

# y_acc 에 학습 셋으로 측정한 정확도의 값을 저장
y_acc=history.history['val_accuracy']

# x값을 지정하고 정확도를 파란색으로, 오차를 빨간색으로 표시
x_len = numpy.arange(len(y_acc))
plt.plot(x_len, y_vloss, "o", c="red", markersize=3)
plt.plot(x_len, y_acc, "o", c="blue", markersize=3)

plt.show()

# early stop
# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

In [ ]:
# -*- coding: utf-8 -*-
# 코드 내부에 한글을 사용가능 하게 해주는 부분입니다.

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

import numpy
import tensorflow as tf
import matplotlib.pyplot as plt

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(3)

# 학습셋, 테스트셋 지정하기
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

# 데이터 전처리
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

# 모델의 설정
model = Sequential()
model.add(Embedding(5000, 100))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

# 모델의 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델의 실행
history = model.fit(x_train, y_train, batch_size=100, epochs=5, validation_data=(x_test, y_test))

# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

# 테스트 셋의 오차
y_vloss = history.history['val_loss']

# 학습셋의 오차
y_loss = history.history['loss']

# 그래프로 표현
x_len = numpy.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
# 모델 저장 폴더 설정
MODEL_DIR = './model0/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)

# 모델 저장 조건 설정
modelpath=MODEL_DIR+"{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)
# 모델 실행 및 저장
history = model.fit(X, Y, validation_split=0.2, epochs=200, batch_size=200, verbose=0, callbacks=[checkpointer,early_stopping_callback])

# y_vloss에 테스트셋으로 실험 결과의 오차 값을 저장
y_vloss=history.history['val_loss']

#### pre-trained word2vec

In [ ]:
import gensim
word2vec_file_path = './model/kow2v/ko.bin'
ko_model = gensim.models.Word2Vec.load(word2vec_file_path)
# ko_model.wv.most_similar("강아지")

vector_size = len(ko_model.wv['아무']) #200
vocab_size  = len(index_to_word) #10000

embedding_matrix = np.zeros((vocab_size, vector_size))
np.shape(embedding_matrix)

def get_vector(word):
    if word in ko_model.wv:
        return ko_model.wv[word]
    else:
        return None

num = 0
not_used = []
for word, i in word_to_index.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.
    else :
        not_used.append(word)
        num+=1

In [ ]:
model1 = Sequential()
e = Embedding(vocab_size, vector_size, weights=[embedding_matrix], input_length=max_len, trainable=False)
model1.add(e)
# model1.add(Flatten())
# model.add(Dropout(0.5))
model1.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
model1.add(MaxPooling1D(pool_size=4))
model1.add(LSTM(55))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
### nlp preprocessing enssential process
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
token = Tokenizer()             # token.word_index , token.document_count , token.word_docs, token.word_index
token.fit_on_texts(docs)   
token.texts_to_sequences(docs)
padded_x = pad_sequences(x, 4)  

#### ways of drawing matplotlib 

In [ ]:
fig = plt.figure(figsize=(15, 5))   
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

plt.figure(figsize=(20,15))
plt.subplot(2,2,1)
plt.subplot(2,2,2)

fig, axs = plt.subplots(3, 1, constrained_layout=True, sharey=True)
axs[0].plot(t1, f(t1), 'o', t2, f(t2), '-')
axs[1].plot(t3, np.cos(2*np.pi*t3), '--')
axs[1].grid(True)  

plt.xlabel('')
plt.ylabel('')
plt.title('jax')

#### Current time (Year ,month,day, Hour, Minute, Second ) 

In [ ]:
import datetime
 
now = datetime.datetime.now()
print(now)          # 2015-04-19 12:11:32.669083 
nowDate = now.strftime('%Y-%m-%d')
print(nowDate)      # 2015-04-19
nowTime = now.strftime('%H:%M:%S')
print(nowTime)      # 12:11:32 
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# 시간 차이 

#1.일,월,주
import datetime
a=datetime.datetime.now()-datetime.timedelta(days=1)#하루전
a=datetime.datetime.now()-datetime.timedelta(hours=1)#한시간전
a=datetime.datetime.now()-datetime.timedelta(weeks=1)#일주일전
#2.달,년
import dateutil.relativedelta import relativedelta
a=datetime.datetime.now()-relativedelta(months=1)#한달전

#### crawling (seleniem) 

In [ ]:
from selenium import webdriver
import time

driver = webdriver.Chrome(r'C:\Users\epsilon\dev\chromedriver_win32\chromedriver.exe') #또는 chromedriver.exe

driver.get(url)
content = driver.find_element_by_xpath(r'//*[@id="powerballLogBox"]/tbody[2]')
docs = content.text

#### missing values by columns 

In [ ]:
# # Missing Values
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

# =================================
# =================================

In [ ]:
def create_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [ ]:
##  onehotencoder
from sklearn import preprocessing    

label_encoder = preprocessing.LabelEncoder()     # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()   # one hot encoder 생성

label_encoder.fit(sse)
se  = label_encoder.transform(sse)
# se = label_encoder.fit_transform(df['pclass'])
from keras.utils import np_utils
Y_train = np_utils.to_categorical(se, 10)

In [41]:
from gensim.models.keyedvectors import Word2VecKeyedVectors
word_vectors = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)
vector = word_vectors['사랑']

UnicodeDecodeError: 'cp949' codec can't decode byte 0x80 in position 0: illegal multibyte sequence

In [ ]:
### model load and save
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드